In [1]:
from decayangle.DecayTopology import TopologyGroup, Node
import numpy as np

# Find the possible the decay topologies
tg = TopologyGroup(0, [1,2,3, 4, 5])
for decay_tree in tg.trees:
    print(decay_tree)

( 0 -> 1, ( (2, 3, 4, 5) -> 2, ( (3, 4, 5) -> 3, ( (4, 5) -> 4, 5 ) ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> 2, ( (3, 4, 5) -> 4, ( (3, 5) -> 3, 5 ) ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> 2, ( (3, 4, 5) -> ( (3, 4) -> 3, 4 ), 5 ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> 3, ( (2, 4, 5) -> 2, ( (4, 5) -> 4, 5 ) ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> 3, ( (2, 4, 5) -> 4, ( (2, 5) -> 2, 5 ) ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> 3, ( (2, 4, 5) -> ( (2, 4) -> 2, 4 ), 5 ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> ( (2, 3) -> 2, 3 ), ( (4, 5) -> 4, 5 ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> 4, ( (2, 3, 5) -> 2, ( (3, 5) -> 3, 5 ) ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> 4, ( (2, 3, 5) -> 3, ( (2, 5) -> 2, 5 ) ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> 4, ( (2, 3, 5) -> ( (2, 3) -> 2, 3 ), 5 ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> ( (2, 4) -> 2, 4 ), ( (3, 5) -> 3, 5 ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> ( (3, 4) -> 3, 4 ), ( (2, 5) -> 2, 5 ) ) )
( 0 -> 1, ( (2, 3, 4, 5) -> ( (2, 3, 4) -> 2, ( (3, 4) -> 3, 4 ) ), 5 ) )
( 0 -> 1, ( (2, 3, 4, 5) -> ( (2, 3, 4) -> 3, (

In [ ]:
def wigner_small_d(theta, j, m1, m2):
    """Calculate Wigner small-d function. Needs sympy.
      theta : angle
      j : spin (in units of 1/2, e.g. 1 for spin=1/2)
      m1 and m2 : spin projections (in units of 1/2)

    :param theta:
    :param j:
    :param m1: before rotation
    :param m2: after rotation

    """
    from sympy import Rational
    from sympy.abc import x
    from sympy.utilities.lambdify import lambdify
    from sympy.physics.quantum.spin import Rotation as Wigner
    j,m1,m2 = int(j),int(m1),int(m2)
    # TODO: check if this is correct (the order of the m1 and m2)
    d = Wigner.d(Rational(j, 2), Rational(m2, 2), Rational(m1, 2), x).doit().evalf()

    return lambdify(x, d, "jax")(theta)

def BWResonance(spin, mass, width):
    """Create a Breit-Wigner resonance function for a given spin.
    Args:
        spin (int): spin quantum number multiplied by 2
    """
    spin_config = list(range(-spin, spin+1, 2))
    def f(s, h0, h1, h2, psi_rf, theta_rf):
        return wigner_small_d(theta_rf, spin, h0, h1 - h2) * np.sqrt(s) / (s - mass**2 + 1j * mass * width)
    
    return f



In [3]:
from math import prod
momenta = { 
    1: np.array([0, 0, -0.9, 1]),
    2: np.array([0, 0.15, 0.4,1]),
    3: np.array([ 0, 0.3, 0.3,1]),
    4: np.array([ 0, 0.25, -0.4,1]),
    5: np.array([ 0, -0.15, -0.3,1])
}

reference_tree = tg.trees[0]
momenta = reference_tree.to_rest_frame(momenta)

amplitudes = {}

# We can define resonances based on the isobar they can appear in 
resonance_lineshapes = {(1, 2): BWResonance(2, 1, 0.1), 
                        (2, 3): lambda s, psi_rf, theta_rf: 1/(s-1+0.1j), 
                        (1, 3): lambda s, psi_rf, theta_rf: 1/(s-1+0.1j)}



for tree in tg.trees:
    final_state_rotations = {
        target:reference_tree.relative_wigner_angles(tree, target, momenta)
        for target in [1, 2, 3, 4, 5]
    }
    isobars = tree.helicity_angles(momenta)
    
    intermediate_state = prod(
            resonance_lineshapes.get(isobar, lambda *args: 0)(tree.nodes[isobar].mass(momenta), 0 ,0) 
            for isobar, angles in isobars.items()
    )
    
    if intermediate_state != 0:
        print(intermediate_state)

        



/home/kai/LHCb/decayangle/src/decayangle/kinematics.py:213: RuntimeWarning: invalid value encountered in divide
  cosine_input = config.backend.where(abs(abs_mom) <= 1e-19, 0, z_component(V) / abs_mom)


(0.19361517555609256-0.039130414088639626j)
